In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn import metrics
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings("ignore")

from scipy.cluster.hierarchy import dendrogram, linkage

from sklearn.cluster import DBSCAN


In [2]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'heartdisease'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
heartdisease_df = pd.read_sql_query('select * from heartdisease',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()


In [3]:
X = heartdisease_df.iloc[:, :13]
y = heartdisease_df.iloc[:, 13]

# Replace missing values (marked by `?`) with a `0`
X = X.replace(to_replace='?', value=0)

# Binarize y so that `1` means heart disease diagnosis and `0` means no diagnosis
y = np.where(y > 0, 0, 1)

In [5]:
X_std=StandardScaler().fit_transform(X)

Apply DBSCAN to the heart disease dataset by trying different values for the eps and min_samples parameters. You'll realize that it's really hard—if not impossible—to get a two-cluster solution using DBSCAN.


In [12]:
es=[.5,1,2,4,8]
for e in es:
    model=DBSCAN(eps=e) 
    clusters=model.fit_predict(X_std)
    print(e, len(np.unique(clusters)))


0.5 1
1 1
2 8
4 2
8 1


In [15]:
mins=[2,4,8,16]
for m in mins:
    model=DBSCAN(min_samples=m) 
    clusters=model.fit_predict(X_std)
    print(m, len(np.unique(clusters)))


2 3
4 1
8 1
16 1


Apply DBSCAN by setting parameters eps=1, min_samples=1, metric="euclidean". Then increase the value of min_samples. When you increase the value of min_samples, how does that affect the number of clusters that DBSCAN identifies?


In [17]:
for m in range(1,10,1):
    model=DBSCAN(eps=1, metric='euclidean', min_samples=m) 
    clusters=model.fit_predict(X_std)
    print(m, len(np.unique(clusters)))

1 294
2 9
3 2
4 1
5 1
6 1
7 1
8 1
9 1


Decreases it

Apply DBSCAN by setting parameters eps=1, min_samples=1, metric="euclidean". Then increase the value of eps. When you increase the value of eps, how does that affect the number of clusters that DBSCAN identifies?


In [24]:
es=np.arange(0.25, 5.0, 0.25)
for e in es:
    model=DBSCAN(eps=e, metric='euclidean', min_samples=1) 
    clusters=model.fit_predict(X_std)
    print(e, len(np.unique(clusters)))

0.25 302
0.5 301
0.75 300
1.0 294
1.25 271
1.5 252
1.75 218
2.0 178
2.25 124
2.5 84
2.75 59
3.0 34
3.25 20
3.5 11
3.75 5
4.0 3
4.25 2
4.5 1
4.75 1


Decreases them